<img src="http://naonedia.fr/wp-content/uploads/2018/10/logoNaonedia.png" style="height:100px">
<h1>
    <center>Expérience Logement</center>
    <center>-</center>
    <center>Analyse des points d'intérêts</center>
</h1>


<h3> Introduction </h3>

Dans ce notebook nous allons voir comment on peut enrichir notre jeu de données.
Pour cela nous allons utiliser <a href="https://github.com/GIScience/openrouteservice/">OpenRouteService</a> et <a href="https://github.com/GIScience/openpoiservice/">OpenPoiService</a>.
Ces deux services nous permettent de localiser des points d'intérêts (POI) à partir d'une géolocalisation.

Grâce à ces données nous allons pouvoir identifier les POIs positifs et négatifs. Ceux-ci peuvent potentiellement améliorer la précision de notre modèle en déterminant de façon plus précise les features discriminantes. Et donc par conséquence estimer le prix d'un  bien plus justement.

<h3> Services </h3>

Pour utiliser les différents, vous pouvez consulter <a href="https://github.com/naonedia/expe_logement_services">notre répertoire</a>. Celui-ci contient tous les services utilisées pour le fonctionnement de notre service. Vous pourrez suivre les étapes d'installation afin de lancer en local vos services si vous avez besoin ré-enrichir les données.

<h3>Informations</h3>

Il est fortement conseillé d'utiliser Python3.x
Si vous ne disposez pas de toutes les librairies requises , il suffit de l'installer en utilisant la commande suivante.
<br>
<code>pip3 install ....</code>

<h3>Authors:</h3>
<cite>Thibault Brocherieux - Ippon Technologies</cite>

In [1]:
import folium
import openrouteservice

import pandas as pd
import numpy as np
import sys

from geojson import MultiPolygon

# Dataset

  * [Demande de valeurs foncières](https://cadastre.data.gouv.fr/dvf#download)

In [2]:
house_dataset = pd.read_pickle('data/datasets/dvf_nantes_cleaned.pkl.xz',  compression='xz')
house_dataset.head()

,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122


## Map visualization

In [3]:
def addMarkerListToMap(coordList, current_map):
    for elem in coordList:
        folium.map.Marker([elem['latitude'],elem['longitude']], # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='red',
                                        icon_color='#cc0000',
                                        icon='pushpin',
                                        prefix='fa',
                                       ),
                 ).add_to(current_map) # Add apartment locations to map
    
    return current_map

# Isochrone

In [4]:
ORS_CLIENT = openrouteservice.Client(base_url='http://localhost:9999/ors')


def buildIsochrone(timeOrDistance, profile, longitude, latitude):
# Request of isochrones with 15 minute footwalk.
    params_iso = {'profile': profile,
                  'intervals': [timeOrDistance], # 900/60 = 15 mins
                  'attributes': ['total_pop'], # Get population count for isochrones
                  'locations': [[longitude,latitude]] # Add apartment coords to request parameters
             }    
    return ORS_CLIENT.isochrones(**params_iso) # Perform isochrone request

In [5]:
map1 = folium.Map(tiles='openstreetmap', location=([47.218371, -1.553621]), zoom_start=12)
iso = buildIsochrone(900,'foot-walking',house_dataset.iloc[0]['longitude'], house_dataset.iloc[0]['latitude'])
folium.features.GeoJson(iso).add_to(map1)

map1

## Points Of Interests

In [6]:
POI_CLIENT = openrouteservice.Client(base_url='http://localhost:5000')

def getPOIForHouse(df, categories_poi, map1, index, timeOrDistance=900, profile='foot-walking'):
    # Retrieve specific house
    row = df.iloc[[index]]

    # Get Isochrones
    iso = buildIsochrone(timeOrDistance, profile,row['longitude'].item(), row['latitude'].item())
    
    # Draw isochrone on map
    folium.features.GeoJson(iso).add_to(map1)
    
    # Center map on house
    map1.location = [row['latitude'].item(),row['longitude'].item()]
    
    # Draw house marker on map
    folium.map.Marker(
        [row['latitude'].item(),row['longitude'].item()],
        icon=folium.Icon(color='lightgray', icon_color='#cc0000', icon='home', prefix='fa'),
        popup="%i m2,  %i pièces\n%i €" % (row['surface_reelle_bati'].item(), row['nombre_pieces_principales'].item(),row['valeur_fonciere'].item())
    ).add_to(map1)
    
    # Common request parameters
    params_poi = {'request': 'pois', 'sortby': 'distance', 'geojson': iso['features'][0]['geometry'] }

    poi = {}
    treatedLocations = []

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        poi[typ] = dict()        
        poi[typ]['geojson'] = POI_CLIENT.places(**params_poi)[0]['features'] # Actual POI request
        
        
        treatedLocations = []
        for elem in poi[typ]['geojson']:
            if elem['geometry']['coordinates'] not in treatedLocations:
                treatedLocations.append(elem['geometry']['coordinates'])
            
                folium.map.Marker(list(reversed(elem['geometry']['coordinates'])), # reverse coords due to weird folium lat/lon syntax
                          icon=folium.Icon(color='blue',
                                            icon_color='#cc0000',
                                            icon='pushpin',
                                            prefix='fa',
                                           ),
                     ).add_to(map1) # Add apartment locations to map
        poi[typ]['geojson'] = treatedLocations

    return map1    

In [7]:
def getPOIGroupForHouse(row, poi_group_id, timeOrDistance=900, profile='foot-walking'):
    iso = {}
    # Get Isochrones
    if pd.Series == type(row) :
        iso = buildIsochrone(timeOrDistance, profile, row['longitude'], row['latitude'])
    else:
        iso = buildIsochrone(timeOrDistance, profile, row['longitude'].item(), row['latitude'].item())

    # Common request parameters
    params_poi = {'request': 'pois', 'filter_category_group_ids': poi_group_id, 'sortby': 'distance', 'geojson': iso['features'][0]['geometry']}
    
    poi = {}
    treatedLocations = []
    
    poi = POI_CLIENT.places(**params_poi)[0]['features'] # Actual POI request
    
    treatedLocations = []
    res = []
    for elem in poi:
        if elem['geometry']['coordinates'] not in treatedLocations:
            treatedLocations.append(elem['geometry']['coordinates'])
            res.append(elem)
        
    return iso, res

def getPOIGroupForHouseMap(row, poi_group_id, map1, timeOrDistance=900, profile='foot-walking'):

    iso, poi = getPOIGroupForHouse(row, poi_group_id, timeOrDistance, profile)
    
    # Draw isochrone on map
    folium.features.GeoJson(iso).add_to(map1)
    
    # Center map on house
    map1.location = [row['latitude'].item(),row['longitude'].item()]
    
    # Draw house marker on map
    folium.map.Marker(
        [row['latitude'].item(),row['longitude'].item()],
        icon=folium.Icon(color='lightgray', icon_color='#cc0000', icon='home', prefix='fa'),
        popup="%i m2,  %i pièces\n%i €" % (row['surface_reelle_bati'].item(), row['nombre_pieces_principales'].item(),row['valeur_fonciere'].item())
    ).add_to(map1)
    
    for elem in poi:
        folium.map.Marker(list(reversed(elem['geometry']['coordinates'])), # reverse coords due to weird folium lat/lon syntax
                          icon=folium.Icon(color='blue', icon_color='#cc0000', icon='pushpin', prefix='fa',),).add_to(map1) # Add apartment locations to map
        
    return map1

# Visualization

In [9]:
POI_CLIENT = openrouteservice.Client(base_url='http://localhost:22222')

map1 = folium.Map(tiles='openstreetmap', location=([47.218371, -1.553621]), zoom_start=12)

categories_poi = {
        'hospital':[843],
    }

getPOIForHouse(house_dataset, categories_poi, map1, 3)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: `item` has been deprecated and will be removed in a future version
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: `item` has been deprecated and will be removed in a future version


In [10]:
map1 = folium.Map(tiles='openstreetmap', location=([47.218371, -1.553621]), zoom_start=12)

getPOIGroupForHouseMap(house_dataset.iloc[[3]], [840], map1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: `item` has been deprecated and will be removed in a future version
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: `item` has been deprecated and will be removed in a future version
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `item` has been deprecated and will be removed in a future version


# Data embedding  
On identifie 5 catégories distinctes. Les nombres associés à chaque catégories ou points d'intérêts est l'identifiant unique.
Celui-ci permet de récupérer les informations via l'API

 * Les transports -> 800
   * arrêt de bus -> 801  
   * arrêt de tram -> 802
   * gare SNCF -> 803
   
* Les parcs -> 810
  * parc -> 811
  * jeu pour enfant -> 812
  
* La vie courante -> 820
  * supermarché -> 821
  * épicier -> 822
  * mairie -> 823
  * pharmacie -> 824
  * établissement social -> 825
  
* Les distractions  -> 830
  * zoo  -> 831
  * gallerie d'art  -> 832
  * musée  -> 833
  * librairie  -> 834
  * centre artistique  -> 835
  * restaurant  -> 836
  * bar  -> 837
  * lieux de culte  -> 838

* Les soins -> 840
  * docteurs -> 841
  * clinic -> 842
  * hospital -> 843

In [11]:
def addPOIs(data):
    index = 0
    for i, row in data.iterrows():
        iso, foot_list_pois = getPOIGroupForHouse(row, [800,810,820,830,840], timeOrDistance=600, profile='foot-walking') # 10 min by foot
        iso, car_list_pois = getPOIGroupForHouse(row, [800,810,820,830,840], timeOrDistance=300, profile='driving-car') # 5 min by car
        
        tmp = {}
        for poi in foot_list_pois:
            for cat, info in poi['properties']['category_ids'].items():
                col_name = 'foot/' + info['category_group'] + '/' + info['category_name']
                if col_name in tmp:
                    tmp[col_name] += 1
                else:
                    tmp[col_name] = 1
        for col_name, number in tmp.items():
            data.loc[i,col_name] = number  
            
        tmp = {}
        for poi in car_list_pois:
            for cat, info in poi['properties']['category_ids'].items():
                col_name = 'car/' + info['category_group'] + '/' + info['category_name']
                if col_name in tmp:
                    tmp[col_name] += 1
                else:
                    tmp[col_name] = 1
                    
        for col_name, number in tmp.items():
            data.loc[i,col_name] = number
        
        index += 1
        sys.stdout.write('\r')
        sys.stdout.write('{}%' .format(round(((index+1) * 100)/data.shape[0],2)))
        sys.stdout.flush()
        
    filter_col = [col for col in data if col.startswith('car') or col.startswith('foot')]
    data[filter_col] = data[filter_col].fillna(value=0)
        
    return data

In [ ]:
enriched_df = addPOIs(house_dataset)